In [ ]:
!pip install numpy
!pip install pandas

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
def get_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)
    return driver

In [3]:
def get_max_page(soup):
    page_no= soup.find_all("a",{ 'class':"num-link mx-0_5 px-0_25 text-gray-darker"})
    pages = []
    for i in page_no:
        pages.append(i.string)
    max_page_number = int(max(pages))
    return max_page_number

In [4]:
def get_page_soup(driver):
    new_page = driver.page_source
    new_soup = BeautifulSoup(new_page)
    return new_soup

In [5]:
def get_car_details(soup):
    car_name = soup.find('h1').get_text()
    car_price = soup.find('div',{'class':"heading-2 mb-0"})
    vin_no = soup.find('span',{'class':'mr-1'}).get_text()
    summery_list = get_summery_list(soup)
    top_features = get_top_features(soup)
    return car_name,car_price,vin_no,summery_list,top_features

In [6]:
def get_summery_list(car_soup):
    summery_section = car_soup.find('div',{'class':'mb-0 max-width text-capitalize pl-1_25 pl-md-0 pl-lg-1_25 row'})
    if(summery_section != None):
        children = summery_section.findChildren("div" , recursive=False)
        grand_childrens=[]
        for child in children:
            grand_children= child.findChildren("div" , recursive=False)
            grand_childrens.append(grand_children)
        summery_list=[]
        for col in grand_childrens:
            for row in col:
                summery_list.append(row.get_text())
        return summery_list
    else:
        return []

In [7]:
def get_top_features(car_soup):
    top_features = car_soup.find('section',{'class':'features-and-specs text-gray-darker'})
    if(top_features != None):
        top_features_children = top_features.findChildren("div" , recursive=False)
        top_features_grand_childrens=[]
        for child in top_features_children:
            grand_children= child.findChildren("div" , recursive=False)
            for grand_element in grand_children:
                top_features_grand_childrens.append(grand_element.findChildren("div" , recursive=False))

        top_features_summery_list=[]
        for col in top_features_grand_childrens:
            for row in range(len(col)):
                key = col[0].get_text()
                value = ''
                if row!=0:
                    value = value + str(col[row].get_text())
                    feature_dict = {key:value}
                    top_features_summery_list.append(feature_dict)
        return top_features_summery_list
    else:
        return []

In [ ]:
base_uri = "https://www.edmunds.com/cars-for-sale-by-owner/"
uri = 'https://www.edmunds.com/inventory/srp.html?inventorytype=used%2Ccpo&pagenumber='
driver_one = get_driver(base_uri)
page_one = driver_one.page_source
soup = BeautifulSoup(page_one)
max_no = get_max_page(soup)
print(max_no)
data_list=[]
for i in range(1,max_no+1):
    uri =uri+str(i)
    driver = get_driver(uri)
    page = driver.page_source
    car_soup = BeautifulSoup(page)
    get_all_cards = car_soup.find_all('a',{'class':'usurp-inventory-card-vdp-link'})
    for i in get_all_cards:
        if i.string !=None:
            link = driver.find_element_by_link_text(i.string)
            # driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            actions = ActionChains(driver)
            actions.move_to_element(link).perform()
            
            link = driver.find_element_by_link_text(i.string)
            link.click()
            linked_page = driver.page_source
            linked_soup = BeautifulSoup(linked_page)
            data_list.append(get_car_details(linked_soup))
            driver.back()
    print(data_list)
df = pd.DataFrame(data_list,columns=['Name','Price','VIN','Vehicle Summary','Top Features & Specs'])
df

555


In [27]:
#from selenium import webdriver
#driver = webdriver.Chrome()
#driver.get("https://www.nytimes.com")
# headlines = driver.find_elements_by_class_name("size-16")
zip_val = driver.find_element_by_name('zip').get_attribute('value')

zip_val

'09622'

In [30]:
elem = driver.find_element_by_class_name('usurp-inventory-card-vdp-link').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".usurp-inventory-card-vdp-link"}
  (Session info: chrome=94.0.4606.71)
